<a href="https://colab.research.google.com/github/ShankarChavan/Gen-AI-LLMs/blob/main/transformers-details/Colab_transformers_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Table of contents**<a id='toc0_'></a>    
- [Positional Encoding](#toc1_)    
- [Multi-Head Attention](#toc2_)    
- [Position-wise Feed-Forward Networks](#toc3_)    
- [_Encoder Layer_](#toc4_)    
- [_Decoder Layer_](#toc5_)    
- [_Transformer Model_](#toc6_)    
- [Prepare Sample Data](#toc7_)    
- [Training the Model](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
print(device)

cuda


<a id='toc1_'></a>
# <a id='toc1_'></a>[Positional Encoding](#toc0_)

Positional Encoding is used to inject the position information of each token in the input sequence. It uses sine and cosine functions of different frequencies to generate the positional encoding.

![1-Input-Embedding](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-3.png?raw=1)
![2-Positional Encoding](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-4.png?raw=1)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

The PositionalEncoding class initializes with input parameters d_model and max_seq_length, creating a tensor to store positional encoding values. The class calculates sine and cosine values for even and odd indices, respectively, based on the scaling factor div_term. The forward method computes the positional encoding by adding the stored positional encoding values to the input tensor, allowing the model to capture the position information of the input sequence.

# <a id='toc2_'></a>[Multi-Head Attention](#toc0_)

The **Multi-Head Attention** mechanism computes the attention between each pair of positions in a sequence. It consists of multiple "attention heads" that capture different aspects of the input sequence.

![3-4-Encoder Layer](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-6.png?raw=1)
![3-Multi-Headed Attention](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-7.png?raw=1)
![Alt text](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/Q_K_V_vector.png?raw=1)
![3.1-Self-Attention](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-8.png?raw=1)
![3.1.2-scores_matrix_mult](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-10.png?raw=1)
![3.1.3-scores_matrix_values](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-11.png?raw=1)
![3.1.4-scores_scaled_matrix](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-12.png?raw=1)
![3.1.5-Softmax_applied_scaled_scores](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-13.png?raw=1)
![3.1.6-Attention_weight_by_values](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-14.png?raw=1)

![3.2.1-Multi-headed-self-attention](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-15.png?raw=1)

![3.2.2-Concat-Linear](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-16.png?raw=1)

![3.3.1-Output-vector](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-17.png?raw=1)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

# <a id='toc3_'></a>[Position-wise Feed-Forward Networks](#toc0_)

![4.-Residual-Conn-layer-norm](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-18.png?raw=1)
![4.1-Residual-LayerNorm-Relu-LayerNorm](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-19.png?raw=1)


In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

The PositionWiseFeedForward class extends PyTorch’s nn.Module and implements a position-wise feed-forward network. The class initializes with two linear transformation layers and a ReLU activation function. The forward method applies these transformations and activation function sequentially to compute the output. This process enables the model to consider the position of input elements while making predictions.

![5-Stacking-of-encoder-layer](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-20.png?raw=1)

# <a id='toc4_'></a>[_Encoder Layer_](#toc0_)

![Alt text](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-21.png?raw=1)

An Encoder layer consists of a Multi-Head Attention layer, a Position-wise Feed-Forward layer, and two Layer Normalization layers.

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

The EncoderLayer class initializes with input parameters and components, including a **MultiHeadAttention module, a PositionWiseFeedForward module, two layer normalization modules, and a dropout layer**. The forward methods computes the encoder layer output by applying self-attention, adding the attention output to the input tensor, and normalizing the result. Then, it computes the position-wise feed-forward output, combines it with the normalized self-attention output, and normalizes the final result before returning the processed tensor.

# <a id='toc5_'></a>[_Decoder Layer_](#toc0_)

![Decoder-Layer](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-23.png?raw=1)

A Decoder layer consists of two **Multi-Head Attention layers, a Position-wise Feed-Forward layer, and three Layer Normalization layers**.

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

The DecoderLayer initializes with input parameters and components such as **MultiHeadAttention modules for masked self-attention and cross-attention, a PositionWiseFeedForward module, three layer normalization modules, and a dropout layer**.

The forward method computes the decoder layer output by performing the following steps:

- Calculate the masked self-attention output and add it to the input tensor, followed by dropout and layer normalization.
- Compute the cross-attention output between the decoder and encoder outputs, and add it to the normalized masked self-attention output, followed by dropout and layer normalization.
- Calculate the position-wise feed-forward output and combine it with the normalized cross-attention output, followed by dropout and layer normalization.
- Return the processed tensor.

These operations enable the decoder to generate target sequences based on the input and the encoder output.

# <a id='toc6_'></a>[_Transformer Model_](#toc0_)

![Alt text](https://github.com/ShankarChavan/Gen-AI-LLMs/blob/main/assets/image-24.png?raw=1)

Merging it all Together

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

The Transformer class combines the previously defined modules to create a complete Transformer model. During initialization, the Transformer module sets up input parameters and initializes various components, including embedding layers for source and target sequences, a PositionalEncoding module, EncoderLayer and DecoderLayer modules to create stacked layers, a linear layer for projecting decoder output, and a dropout layer.

The generate_mask method creates binary masks for source and target sequences to ignore padding tokens and prevent the decoder from attending to future tokens.

The forward method computes the Transformer model’s output through the following steps:

1. Generate source and target masks using the generate_mask method.
2. Compute source and target embeddings, and apply positional encoding and dropout.
3. Process the source sequence through encoder layers, updating the enc_output tensor.
4. Process the target sequence through decoder layers, using enc_output and masks, and updating the dec_output tensor.
5. Apply the linear projection layer to the decoder output, obtaining output logits.

These steps enable the Transformer model to process input sequences and generate output sequences based on the combined functionality of its components.

# <a id='toc7_'></a>[Prepare Sample Data](#toc0_)

In this example, we will create a toy dataset for demonstration purposes. In practice, you would use a larger dataset, preprocess the text, and create vocabulary mappings for source and target languages.

In [ ]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

# <a id='toc8_'></a>[Training the Model](#toc0_)
Now we’ll train the model using the sample data. In practice, you would use a larger dataset and split it into training and validation sets.

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.670790672302246
Epoch: 2, Loss: 8.542296409606934
Epoch: 3, Loss: 8.476363182067871
Epoch: 4, Loss: 8.420205116271973
Epoch: 5, Loss: 8.362591743469238
Epoch: 6, Loss: 8.282110214233398
Epoch: 7, Loss: 8.20651912689209
Epoch: 8, Loss: 8.123579025268555
Epoch: 9, Loss: 8.042201042175293
Epoch: 10, Loss: 7.960916519165039
Epoch: 11, Loss: 7.882750988006592
Epoch: 12, Loss: 7.794997692108154
Epoch: 13, Loss: 7.712817192077637
Epoch: 14, Loss: 7.625358581542969
Epoch: 15, Loss: 7.548154830932617
Epoch: 16, Loss: 7.461191654205322
Epoch: 17, Loss: 7.3775200843811035
Epoch: 18, Loss: 7.294585704803467
Epoch: 19, Loss: 7.211431980133057
Epoch: 20, Loss: 7.137082099914551
Epoch: 21, Loss: 7.0622029304504395
Epoch: 22, Loss: 6.984019756317139
Epoch: 23, Loss: 6.898898124694824
Epoch: 24, Loss: 6.827769756317139
Epoch: 25, Loss: 6.747014999389648
Epoch: 26, Loss: 6.681588649749756
Epoch: 27, Loss: 6.599752426147461
Epoch: 28, Loss: 6.534604072570801
Epoch: 29, Loss: 6.463239669